In [4]:
!mkdir .kaggle
import json
token = {"username":"bgudltaldanny","key":"65b62f1a36c14c834b4e88037463a4f8"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

mkdir: cannot create directory ‘.kaggle’: File exists


In [3]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


!kaggle competitions download -c home-depot-product-search-relevance
!unzip home-depot-product-search-relevance.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/train.csv.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/test.csv.zip -d home-depot-product-search-relevance/

mkdir: cannot create directory ‘.kaggle’: File exists
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl
 59% 41.0M/69.4M [00:00<00:00, 37.4MB/s]
100% 69.4M/69.4M [00:00<00:00, 108MB/s] 
Archive:  home-depot-product-search-relevance.zip
  inflating: home-depot-product-search-relevance/attributes.csv.zip  
  inflating: home-depot-product-search-relevance/product_descriptions.csv.zip  
  inflating: home-depot-product-search-relevance/relevance_instructions.docx  
  inflating: home-depot-product-search-relevance/sample_submission.csv.zip  
  inflating: home-depot-product-search-relevance/test.csv.zip  
  inflating: home-depot-product-search-relevance/train.csv.zip  
Archive:  home-depot-product-search-relevance/train.csv.zip
  inflating: home-depot-product-search-